In [2]:
from pymongo import MongoClient
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import string
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import copy
import pickle
import os

In [3]:
summary = np.load("summary.npy")
original_summary = np.load("original.npy")

The user can input multiple things.
1 - doi of the article they want to find (from doi we extract summary, preprocess then serves as input to vector) -- figure out how to query this later
2 - summary of article (preprocess summary then find most similar article)
3 - title if in db (use corresponding summary, preprocess then pass through model)

If the user inputs the doi

In [ ]:
#BASE_URL = 'http://export.arxiv.org/api/query?search_query=all:Machine%20Learning'

If the user inputs summary

In [ ]:
#user_input = test_data[-3][0] # Already preprocessed, but simply pass through function

If the user inputs title:
    - if title present in db get summary, preprocess then pass through model
    - if title not present in db then preprocess title and pass through model

In [4]:
vectorizer = 'tfidf_vectorized.pkl'
tfidf_model = 'tfidf.pkl'

In [5]:
summary_full = [summary[index][0] for index in range(len(summary))]

In [6]:
if os.path.exists(vectorizer) and os.path.exists(tfidf_model):
    with open(vectorizer, "rb") as f:
        tfidf_vectorizer = pickle.load(f)
    with open(tfidf_model, "rb") as f:
        tfidf_mat = pickle.load(f)
else:
    tfidf_vectorizer = TfidfVectorizer(lowercase=True, stop_words='english')
    tfidf_mat = tfidf_vectorizer.fit_transform(summary_full)
    with open(vectorizer, "wb") as f:
        pickle.dump(tfidf_vectorizer, f)
    with open(tfidf_model, "wb") as f:
        pickle.dump(tfidf_mat, f)
    print("saved to cache")

saved to cache


Here we get the user input to pass through model

In [7]:
new_tfidf = tfidf_vectorizer.transform([summary_full[-1][0]])

In [10]:
similarities = cosine_similarity(new_tfidf, tfidf_mat)
most_similar = similarities.argmax()
score = similarities.max()

In [11]:
similarities_np = np.array(similarities[0])
highest = np.argpartition(similarities_np, -5)[-5:]

We want to return the title for the objects with the 5 highest scores (this can be changed by the user)

In [12]:
clean_arr = np.array([[high, similarities_np[high]] for high in highest])
clean_arr = clean_arr[clean_arr[:,1].argsort()][::-1]

In [13]:
client = MongoClient("mongodb://localhost:27017/")
db = client["Arxiv"]
collection = db["Arxiv Papers"]

This is what we return to the user, namely, the title of the respective articles

In [14]:
summary_top_5 = np.array(original_summary)[highest]
for summarized in summary_top_5:
    query = {"summary" : summarized[0]}
    found = collection.find(query)
    for document in found:
        print(document['title'], "\n")

streaming adaptive nonparametric variational autoencoder 

using learned optimizers to make models robust to input noise 

risks from learned optimization in advanced machine learning systems 

sparsesense: human activity recognition from highly sparse sensor
  data-streams using set-based neural networks 

self-supervised contrastive learning for eeg-based sleep staging 

